In [5]:
import pickle


import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import mlflow



ImportError: cannot import name 'asarray' from 'numpy._core' (/Users/steffi/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/_core/__init__.py)

In [ ]:
from sklearn.pipeline import make_pipeline

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

RUN_ID = os.getenv("RUN_ID", "dee4d04c51914d708c3350051843826f")
logged_model = f"s3://mlflow-artifact-stef/1/{RUN_ID}/artifacts/model"
# logged_model = f"runs:/{RUN_ID}/model"
model = mlflow.pyfunc.load_model(logged_model)

2025/06/07 13:32:06 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.1.1, required: cloudpickle==2.2.1)
 - numpy (current: 2.2.6, required: numpy==1.26.4)
 - scipy (current: 1.15.3, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/06/07 13:32:06 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.9.6`, differs from the version of Python that is currently running, `Python 3.11.11`, and may be incompatible
/Users/steffi/Documents/MLOps-Zoomcamp/mlops-zoomcamp/venv/lib/python3.11/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

'runs:/dee4d04c51914d708c3350051843826f/model'

In [ ]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [ ]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [ ]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 15.136777093556063


In [ ]:
from mlflow.tracking import MlflowClient


In [ ]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv

DictVectorizer()